In [4]:
import csv
import pandas as pd
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="Census Data",
    user="postgres",
    password="postgres"
)
cur = conn.cursor()

# Function to get the next available ID for a table
def get_next_id(table, id_column):
    cur.execute(f'SELECT COALESCE(MAX("{id_column}"), 0) + 1 FROM {table}')
    return cur.fetchone()[0]

# File path to the dataset
data_file = '/Users/adityasrivatsav/Documents/sql/sql_project/census_income_project/data/adult.data'

# Load the dataset
columns = [
    'age', 'workclass', 'fnlwgt', 'education_level', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'sex',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income_class'
]
df = pd.read_csv(data_file, header=None, names=columns, na_values=' ?')

# Handle missing values by filling them
df.fillna({
    'workclass': 'Unknown',
    'education_level': 'Unknown',
    'occupation': 'Unknown',
    'relationship': 'Unmarried',
    'native_country': 'United-States',
    'income_class': '<=50K'
}, inplace=True)

# Ensure numeric columns have default values
numeric_columns = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
for col in numeric_columns:
    df[col] = df[col].fillna(0).astype(int)

# Drop rows with any remaining missing values
df_cleaned = df.dropna()

# Iterate over the cleaned dataset and insert data into the database
for _, row in df_cleaned.iterrows():
    try:
        age = int(row['age'])
        fnlwgt = int(row['fnlwgt'])
        sex = row['sex'] or 'Unknown'
        hours_per_week = int(row['hours_per_week'])
        native_country = row['native_country'] or 'Unknown'
        workclass = row['workclass'] or 'Unknown'
        occupation = row['occupation'] or 'Unknown'
        education_level = row['education_level'] or 'Unknown'
        education_num = int(row['education_num'])
        marital_status = row['marital_status'] or 'Unknown'
        relationship_status = row['relationship'] or 'Unmarried'
        capital_gain = int(row['capital_gain'])
        capital_loss = int(row['capital_loss'])
        income_class = row['income_class'].strip() or '<=50K'

        # Insert into Individuals table
        individual_id = get_next_id('Individuals', 'individual_id')
        cur.execute("""
            INSERT INTO Individuals (individual_id, age, fnlwgt, sex, hours_per_week, native_country)
            VALUES (%s, %s, %s, %s, %s, %s);
        """, (individual_id, age, fnlwgt, sex, hours_per_week, native_country))

        # Insert into Employment table
        employment_id = get_next_id('Employment', 'employment_id')
        cur.execute("""
            INSERT INTO Employment (employment_id, individual_id, capital_gain, capital_loss)
            VALUES (%s, %s, %s, %s);
        """, (employment_id, individual_id, capital_gain, capital_loss))

        # Insert into JobDetails table
        cur.execute("""
            INSERT INTO JobDetails (individual_id, workclass, occupation)
            VALUES (%s, %s, %s);
        """, (individual_id, workclass, occupation))

        # Insert into Education table
        education_id = get_next_id('Education', 'education_id')
        cur.execute("""
            INSERT INTO Education (education_id, individual_id)
            VALUES (%s, %s);
        """, (education_id, individual_id))

        # Insert into EducationDetails table
        cur.execute("""
            INSERT INTO EducationDetails (individual_id, education_level, education_num)
            VALUES (%s, %s, %s);
        """, (individual_id, education_level, education_num))

        # Insert into Marital table
        marital_id = get_next_id('Marital', 'marital_id')
        cur.execute("""
            INSERT INTO Marital (marital_id, individual_id)
            VALUES (%s, %s);
        """, (marital_id, individual_id))

        # Insert into RelationshipDetails table
        cur.execute("""
            INSERT INTO RelationshipDetails (individual_id, marital_status, relationship_status)
            VALUES (%s, %s, %s);
        """, (individual_id, marital_status, relationship_status))

        # Insert into Income table
        income_id = get_next_id('Income', 'income_id')
        cur.execute("""
            INSERT INTO Income (income_id, individual_id)
            VALUES (%s, %s);
        """, (income_id, individual_id))

        # Insert into IncomeDetails table
        cur.execute("""
            INSERT INTO IncomeDetails (individual_id, income_class)
            VALUES (%s, %s);
        """, (individual_id, income_class))

        # Commit after each row to avoid data loss in case of failure
        conn.commit()

    except Exception as e:
        # Rollback in case of error and log the issue
        conn.rollback()
        print(f"Error processing row {row.to_dict()}: {e}")

# Close the cursor and connection
cur.close()
conn.close()